<a href="https://colab.research.google.com/github/rianachatterjee04/GenAssist/blob/main/YOLOv8_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi


/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
!pip install ultralytics


Importing YOLOv8

In [ ]:
from ultralytics import YOLO
import os
from IPython.display import display, Image
from IPython import display
display.clear_output()
!yolo mode=checks

Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
             ^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/ultralytics/cfg/__init__.py", line 911, in entrypoint
    raise ValueError(f"Invalid 'mode={mode}'. Valid modes are {MODES}.\n{CLI_HELP_MSG}")
ValueError: Invalid 'mode=<module 'ultralytics.utils.checks' from '/usr/local/lib/python3.11/dist-packages/ultralytics/utils/checks.py'>'. Valid modes are {'track', 'train', 'benchmark', 'predict', 'val', 'export'}.

    Arguments received: ['yolo', 'mode=checks']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

        Where   TASK (optional) is one of {'detect', 'obb', 'pose', 'classify', 'segment'}
                MODE (required) is one of {'track', 'train', 'benchmark', 'predict', 'val', 'export'}
                ARGS (optional) are any number of custom 'arg=value' pairs like 'imgsz=320' that override defaults.
            

Train YOLOv8 Model on Custom Dataset

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="YOUR API KEY")
project = rf.workspace("nicolai-hoirup-nielsen").project

Installing Dependencies

In [ ]:
# Install yt-dlp for downloading YouTube videos
!pip install yt-dlp

# Install OpenCV for video frame extraction
!pip install opencv-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 47.4 MB/s eta 0:00:00


Downloading Youtube Videos

In [ ]:
import os

os.makedirs("videos", exist_ok = True)

VIDEO_URL = "https://www.youtube.com/watch?v=EVrqJkbV1bw"
!yt-dlp -o "videos/%(new_york)s.%(ext)s" {VIDEO_URL}

!ls videos

[youtube] Extracting URL: https://www.youtube.com/watch?v=RFb6wbsffY0
[youtube] RFb6wbsffY0: Downloading webpage
[youtube] RFb6wbsffY0: Downloading tv player API JSON
[youtube] RFb6wbsffY0: Downloading ios player API JSON
[youtube] RFb6wbsffY0: Downloading player 6e1dd460
[youtube] RFb6wbsffY0: Downloading m3u8 information
[info] RFb6wbsffY0: Downloading 1 format(s): 315+251
[download] Destination: videos/NA.f315.webm
[download] 100% of    1.07GiB in 00:00:41 at 26.12MiB/s
[download] Destination: videos/NA.f251.webm
[download] 100% of    4.66MiB in 00:00:00 at 30.76MiB/s
[Merger] Merging formats into "videos/NA.webm"
Deleting original file videos/NA.f315.webm (pass -k to keep)
Deleting original file videos/NA.f251.webm (pass -k to keep)
NA.webm


In [ ]:
!ls videos

NA.webm


Extracting frames from the video

In [ ]:
import cv2

# Path to video file
video_path = "videos/NA.webm"  # Replace with the actual video filename
output_dir = "frames"
os.makedirs(output_dir, exist_ok=True)

# Open the video file
vidcap = cv2.VideoCapture(video_path)
success, image = vidcap.read()
frame_rate = 30  # Extract one frame every 30 frames (adjust based on your needs)
count = 0

# Loop through frames
while success:
    if int(vidcap.get(cv2.CAP_PROP_POS_FRAMES)) % frame_rate == 0:
        frame_name = os.path.join(output_dir, f"frame{count}.jpg")
        cv2.imwrite(frame_name, image)  # Save frame as JPEG
        print(f"Saved: {frame_name}")
        count += 1
    success, image = vidcap.read()

print(f"Extracted {count} frames to {output_dir}")

Saved: frames/frame0.jpg
Saved: frames/frame1.jpg
Saved: frames/frame2.jpg
Saved: frames/frame3.jpg
Saved: frames/frame4.jpg
Saved: frames/frame5.jpg
Saved: frames/frame6.jpg
Saved: frames/frame7.jpg
Saved: frames/frame8.jpg
Saved: frames/frame9.jpg
Saved: frames/frame10.jpg
Saved: frames/frame11.jpg
Saved: frames/frame12.jpg
Saved: frames/frame13.jpg
Saved: frames/frame14.jpg
Saved: frames/frame15.jpg
Saved: frames/frame16.jpg
Saved: frames/frame17.jpg
Saved: frames/frame18.jpg
Saved: frames/frame19.jpg
Saved: frames/frame20.jpg
Saved: frames/frame21.jpg
Saved: frames/frame22.jpg
Saved: frames/frame23.jpg
Saved: frames/frame24.jpg
Saved: frames/frame25.jpg
Saved: frames/frame26.jpg
Saved: frames/frame27.jpg
Saved: frames/frame28.jpg
Saved: frames/frame29.jpg
Saved: frames/frame30.jpg
Saved: frames/frame31.jpg
Saved: frames/frame32.jpg
Saved: frames/frame33.jpg
Saved: frames/frame34.jpg
Saved: frames/frame35.jpg
Saved: frames/frame36.jpg
Saved: frames/frame37.jpg
Saved: frames/frame38.

Organize the dataset

In [ ]:
import shutil

# Create dataset folder structure
os.makedirs("dataset/images/train", exist_ok=True)
os.makedirs("dataset/images/val", exist_ok=True)
os.makedirs("dataset/images/test", exist_ok=True)

# Move extracted frames into the train folder for now
for file in os.listdir(output_dir):
    shutil.move(os.path.join(output_dir, file), "dataset/images/train")

print("Dataset organized!")


Dataset organized!


Downloading directories

In [ ]:
import os
import shutil

# Define the target folder
target_folder = "YOLO_v8"

# Create the target folder
os.makedirs(target_folder, exist_ok=True)

# Directories to consolidate
directories = ["dataset", "frames", "sample_data", "videos"]

# Move each directory into the target folder
for dir_name in directories:
    if os.path.exists(dir_name):
        shutil.move(dir_name, os.path.join(target_folder, dir_name))
        print(f"Moved: {dir_name} -> {os.path.join(target_folder, dir_name)}")
    else:
        print(f"Directory '{dir_name}' not found, skipping...")

print(f"All directories consolidated into '{target_folder}'.")

# List the contents of the YOLO_v8 folder to verify
print("\nContents of 'YOLO_v8':")
!ls YOLO_v8


Moved: dataset -> YOLO_v8/dataset
Moved: frames -> YOLO_v8/frames
Moved: sample_data -> YOLO_v8/sample_data
Moved: videos -> YOLO_v8/videos
All directories consolidated into 'YOLO_v8'.

Contents of 'YOLO_v8':
dataset  frames  sample_data  videos


In [ ]:
!zip -r YOLO_v8.zip YOLO_v8


  adding: YOLO_v8/ (stored 0%)
  adding: YOLO_v8/sample_data/ (stored 0%)
  adding: YOLO_v8/sample_data/README.md (deflated 39%)
  adding: YOLO_v8/sample_data/anscombe.json (deflated 83%)
  adding: YOLO_v8/sample_data/mnist_train_small.csv (deflated 88%)
  adding: YOLO_v8/sample_data/mnist_test.csv (deflated 88%)
  adding: YOLO_v8/sample_data/california_housing_test.csv (deflated 76%)
  adding: YOLO_v8/sample_data/california_housing_train.csv (deflated 79%)
  adding: YOLO_v8/videos/ (stored 0%)
  adding: YOLO_v8/videos/NA.webm (deflated 0%)
  adding: YOLO_v8/dataset/ (stored 0%)
  adding: YOLO_v8/dataset/images/ (stored 0%)
  adding: YOLO_v8/dataset/images/test/ (stored 0%)
  adding: YOLO_v8/dataset/images/val/ (stored 0%)
  adding: YOLO_v8/dataset/images/train/ (stored 0%)
  adding: YOLO_v8/dataset/images/train/frame381.jpg (deflated 0%)
  adding: YOLO_v8/dataset/images/train/frame465.jpg (deflated 1%)
  adding: YOLO_v8/dataset/images/train/frame475.jpg (deflated 1%)
  adding: YOLO_v8

In [ ]:
from google.colab import files
files.download("YOLO_v8.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Copy the file to your Google Drive
!cp YOLO_v8.zip /content/drive/MyDrive/YOLO_v8.zip

print("File has been copied to Google Drive in 'My Drive' as 'YOLO_v8.zip'")


Mounted at /content/drive
File has been copied to Google Drive in 'My Drive' as 'YOLO_v8.zip'


In [ ]:
!du -h YOLO_v8.zip


2.5G	YOLO_v8.zip


Importing directly into roboflow (not working)

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="CDDO05uGHWiWIGcIMvjI")

workspace = rf.workspace("correct_workspace_name")  # Replace with the verified workspace name
for project in workspace.projects():
    print("Project Name:", project.name)

loading Roboflow workspace...


RoboflowError: {
    "error": {
        "message": "Unsupported get request. Workspace with ID \"correct_workspace_name\" does not exist or cannot be loaded due to missing permissions.",
        "status": 404,
        "type": "GraphMethodException",
        "hint": "You can see your available workspaces by issuing a GET request to /workspaces"
    }
}

Upload Roboflow Dataset

In [ ]:
# Upload your dataset ZIP file
from google.colab import files
uploaded = files.upload()

# Unzip the dataset
!unzip dataset.zip -d dataset


Train Model

In [ ]:


# Import YOLO and train the model
from ultralytics import YOLO

# Load a pretrained YOLOv8 model
model = YOLO('yolov8n.pt')  # Use 'yolov8n.pt' (nano) for faster training

# Train the model
model.train(data='dataset/data.yaml', epochs=20, imgsz=640)


Visualize the Results

In [ ]:
from IPython.display import Image

# Confusion matrix
Image(filename='runs/detect/train/confusion_matrix.png')

# Loss and metrics graphs
Image(filename='runs/detect/train/results.png')


In [ ]:
!ls


dataset  frames  sample_data  videos
